## Read GCOEFF.txt file

In [1]:
import os 
import resource
import numpy as np

#os.chdir('example_wavecar_InSe/')

file_name = 'example_wavecar_InSe/GCOEFF.txt'
print(f'File Size is {os.stat(file_name).st_size / (1024 * 1024)} MB')

txt_file = open(file_name)
lines = txt_file.readlines()

#Read info
real_lat = []
rec_lat = []
num_kpts = int(lines[1].split()[0])
num_bands = int(lines[2].split()[0])
for nn in [3,4,5]:
    real_lat.append([float(x) for x in lines[nn].split()])
for nn in [6,7,8]:
    rec_lat.append([float(x) for x in lines[nn].split()])    

#Main loop read fourier components
kpts = []
Eigenvals = []
occupation = []
G_values=[]
PW_coeff=[]
num_plane_waves = []
kpt_line = [9]
band_line = [10]
ev_line = [11]
for nk in range(num_kpts):
    kpts.append([float(x) for x in lines[kpt_line[-1]].split()])
    for nb in range(num_bands):
        num_plane_waves.append(int(lines[band_line[nk*num_bands+nb]].split()[1]))
        Eigenvals.append([nk,complex(float(lines[ev_line[nk*num_bands+nb]].split()[1]),float(lines[ev_line[nk*num_bands+nb]].split()[3]))])
        occupation.append([nk,float(lines[ev_line[nk*num_bands+nb]].split()[5])])
        for npw in range(num_plane_waves[nk*num_bands+nb]):
            G_values.append([nk,nb, [int(x) for x in lines[ev_line[nk*num_bands+nb]+1+npw].split()[:3]  ]  ])
            PW_coeff.append([nk,nb, complex(float(lines[ev_line[nk*num_bands+nb]+1+npw].split()[4]), float(lines[ev_line[nk*num_bands+nb]+1+npw].split()[6]) )   ])
        if nb==num_bands-1:
            band_line.append( band_line[nk*num_bands+nb]+num_plane_waves[nk*num_bands+nb]+3)
            ev_line.append( ev_line[nk*num_bands+nb]+num_plane_waves[nk*num_bands+nb]+3)
        else:
            band_line.append( band_line[nk*num_bands+nb]+num_plane_waves[nk*num_bands+nb]+2)
            ev_line.append( ev_line[nk*num_bands+nb]+num_plane_waves[nk*num_bands+nb]+2)
    kpt_line.append(band_line[nk*num_bands+nb]+num_plane_waves[nk*num_bands+nb]+2)
    

txt_file.close()
print(f'Number of Lines in the file is {len(lines)}')
print('Peak Memory Usage =', resource.getrusage(resource.RUSAGE_SELF).ru_maxrss)
print('User Mode Time =', resource.getrusage(resource.RUSAGE_SELF).ru_utime)
print('System Mode Time =', resource.getrusage(resource.RUSAGE_SELF).ru_stime)

File Size is 205.31703758239746 MB
Number of Lines in the file is 3777213
Peak Memory Usage = 1852044
User Mode Time = 12.831818
System Mode Time = 2.655207


### Calculate charge density

#### A few new things learned
( np.sum(np.array) takes 3-4 times more time than sum(np.array)) 

Try to use Real and Imag parts separately: that way do dont have to use complex math everytime

In [6]:
### Read OUTCAR
from pymatgen.io.vasp import outputs
import cmath

[NGXF , NGYF , NGZF]=outputs.Outcar('example_wavecar_InSe/OUTCAR').ngf

### discretize real lattice into NGXF x NGYF x NGZF
##adding only n th band to generate psin
iband = 10
psin = np.zeros((NGXF , NGYF , NGZF, num_bands),dtype=np.complex_)
density = np.zeros((NGXF , NGYF , NGZF, num_bands))
kplusG = []

for nn, npw in enumerate(PW_coeff):
    kplusG.append(list(sum(kpts[npw[0]] , np.dot(rec_lat,G_values[nn][2]) ) ) )
      
for i in range(NGXF):
    for j in range(NGYF):
        for k in range(NGZF):
            print(k)
            r = [i/NGXF, j/NGYF, k/NGZF]
            psir = [[] for _ in range(num_bands)] 
            for nn, npw in enumerate(PW_coeff):
                psir[npw[1]].append( npw[2]*cmath.exp( np.dot(kplusG[nn] , r ) *1j ) )
            
            psin[i,j,k,:] = [sum(n) for n in psir]
            density[i,j,k,:] = np.multiply(np.conj(psin[i,j,k,:]), psin[i,j,k,:])
                    

                


0


/tmp/ipykernel_237920/1991702696.py:27: ComplexWarning: Casting complex values to real discards the imaginary part
  density[i,j,k,:] = np.multiply(np.conj(psin[i,j,k,:]), psin[i,j,k,:])


1
2


KeyboardInterrupt: 

In [17]:
timeit [np.sum(n) for n in psir]

81.9 ms ± 1.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [102]:
#### CHATGPT alternative
import numpy as np

# Assuming required variables are preloaded: NGXF, NGYF, NGZF, num_bands, PW_coeff, kpts, rec_lat, G_values

# Initialize psin with zeros
psin = np.zeros((NGXF, NGYF, NGZF, num_bands), dtype=np.complex64)

# Precompute k + G vectors
kplusG = np.array([kpts[npw[0]] + np.dot(rec_lat, G_values[nn][2]) for nn, npw in enumerate(PW_coeff)])

# Generate real-space grid
x = np.linspace(0, 1, NGXF, endpoint=False)
y = np.linspace(0, 1, NGYF, endpoint=False)
z = np.linspace(0, 1, NGZF, endpoint=False)
X, Y, Z = np.meshgrid(x, y, z, indexing='ij')

# Combine the real-space grid into a single array of shape (NGXF, NGYF, NGZF, 3)
r_grid = np.stack((X, Y, Z), axis=-1)

# Reshape the grid to a 2D array of shape (NGXF*NGYF*NGZF, 3) for easier computation
r_grid_flat = r_grid.reshape(-1, 3)

# Compute phase factors exp[i * (k + G) · r] for all PW_coeff entries and grid points
phase_factors = np.exp(1j * np.dot(r_grid_flat, kplusG.T))  # Shape: (NGXF*NGYF*NGZF, len(PW_coeff))

# Initialize psir array with zeros
psir = np.zeros((len(PW_coeff), num_bands), dtype=np.complex_)

# Group contributions to psir by band
for nn, npw in enumerate(PW_coeff):
    psir[nn, npw[1]] += npw[2] * phase_factors[:, nn]

# Sum over bands
psin_flat = psir.sum(axis=0)

# Reshape psin back to grid shape
psin = psin_flat.reshape(NGXF, NGYF, NGZF, num_bands)




MemoryError: Unable to allocate 53.4 TiB for an array with shape (1944000, 3776616) and data type float64